# Evolving quantum circuits - Example project

This notebook shows how you may use the quantum-circuit-evolver to find quantum circuits fittet to a 1D Von Neumann neigbourhood for cellular automata. This guide will go through the process of initialising the needed parameters, creating chromosomes generations and in the end simulationg them on the qiskit simulator to see the quantum results.

First we import the quantum_circuit_evolver library. __Note!__ You need Qiskit for the library to work.

In [2]:
from QuantumCircuitEvolver.quantum_circuit_evolver import *

Then we set up the evovlers parameters. 
- **gates**: The number of quantum gates per circuit.
- **chromosomes**: The number of chromosomes or solutions you want in each generation.
- **generation**: The number of generations you want to evolve before terminating.
- **desired_chance_of_one**: A list of the eight probabilities for the eight CA initial states you want the circuit to generate. There are some example probability list commented. Remove the comment to test them!

In [3]:
gates = 5
chromosomes = 10
generations = 20
gate_types = ['cx', 'h', 'x']  # possible gates: # h, cx, x, y, z, swap, rzz, rxx, toffoli

#desired_chance_of_one = [1, 0, 1, 0, 0, 1, 0, 1]
desired_chance_of_one = [0.5, 0.7, 0.4, 0.0, 0.2, 0.7, 0.1, 0.9]
# desired_chance_of_one = [1.0, 0.5, 1.0, 0.0, 0.0, 1.0, 0.5, 0.0]

In [4]:
# Generate initial generation of chromosomes
init_gen = Generation(chromosomes, gates)
init_gen.create_initial_generation(gate_types)
init_gen.print_chromosomes()



Chromosomes: 
[0, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 0, 2, 2, 2]
[1, 2, 1, 0, 1, 0, 0, 1, 0, 0, 1, 2, 1, 2, 2]
[2, 0, 1, 1, 1, 2, 1, 1, 2, 0, 1, 0, 0, 1, 0]
[2, 1, 2, 0, 1, 0, 2, 2, 2, 2, 0, 2, 0, 1, 0]
[1, 2, 2, 2, 0, 2, 0, 1, 0, 0, 0, 2, 0, 2, 1]
[0, 2, 0, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 2, 0]
[1, 0, 2, 1, 2, 1, 0, 1, 0, 1, 2, 0, 1, 1, 0]
[2, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 2, 0, 1, 0]
[1, 0, 1, 2, 0, 1, 1, 0, 2, 0, 2, 1, 1, 0, 2]
[0, 1, 0, 1, 1, 2, 2, 1, 1, 0, 0, 2, 0, 0, 1]




Then we check all the chromosomes fitness and print the results.

In [5]:
init_gen.run_generation(desired_chance_of_one)

print("Fitness for best chromosome: " + str(init_gen.get_best_fitness()) + "\n"
      + "Best chromosome: \n" + str(init_gen.get_best_chromosome()))
print("\n")

Fitness for best chromosome: 2.115
Best chromosome: 
[2, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 2, 0, 1, 0]




Before we can start the evolution process, we need to declare some container values fopr the best performing chromosome.

In [6]:
# Final value placeholders
current_chromosome = init_gen.get_best_chromosome()
best_chromosome = current_chromosome
final_fitness = init_gen.get_best_fitness()

Then we create a loop for the number of generations, and every time find the best chromosome in the generation to be a parent for the next generation.

In [7]:
# Mutation loop
for gen in range(0, generations):

    # Mutate next generation of chromosomes
    next_gen = Generation(chromosomes, gates)
    next_gen.create_mutated_generation(current_chromosome)

    # Check every Chromosome's fitness
    next_gen.run_generation(desired_chance_of_one)

    current_fitness = next_gen.get_best_fitness()
    current_chromosome = next_gen.get_best_chromosome()
    
    # Print generation best result
    print("Fitness for best mutated chromosome in mutation " + str(gen + 1) + ": "
          + str(current_fitness) + "\n"
          + "Best mutated chromosome:\n" + str(next_gen.get_best_chromosome()))
    print("\n")

    # Check if there is a new_list best chromosome

    if final_fitness > current_fitness:
        final_fitness = current_fitness
        best_chromosome = current_chromosome
        print("New best!")
        
        print("\n")
    if current_fitness < 0.01: # Stop if high enough fitness is reached
        break
    print("------------------------------------------------------------------------------")


Fitness for best mutated chromosome in mutation 1: 2.062
Best mutated chromosome:
[2, 1, 0, 1, 0, 2, 2, 0, 1, 2, 1, 0, 0, 1, 0]


New best!


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 2: 2.013
Best mutated chromosome:
[1, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 0, 2, 1, 1]


New best!


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 3: 2.074
Best mutated chromosome:
[1, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 0, 2, 2, 0]


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 4: 2.074
Best mutated chromosome:
[1, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 0, 0, 1, 0]


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 5: 2.0489999999999995
Best mutated chromosome:
[1, 0, 0, 1, 2, 1, 0, 1, 0, 2, 2, 0, 0, 1, 0]

Last, let's print the result of the best found ciurcuit and visualize it.

In [8]:
print("Best fitness found: " + str(final_fitness))
print("Best chromosome found: " + str(best_chromosome))
print("\n")
circuit = Circuit(best_chromosome)
circuit.print_ca_outcomes(desired_chance_of_one)


circuit.generate_circuit()
circuit.draw()

Best fitness found: 2.013
Best chromosome found: [1, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 0, 2, 1, 1]


Initial State | Desired outcome | Actual outcome  | Difference
[0, 0, 0]              0.5               0.504           0.004
[0, 0, 1]              0.7               0.509           0.191
[0, 1, 0]              0.4               0.487           0.087
[0, 1, 1]              0.0               0.520           0.520
[1, 0, 0]              0.2               0.491           0.291
[1, 0, 1]              0.7               0.477           0.223
[1, 1, 0]              0.1               0.513           0.413
[1, 1, 1]              0.9               0.489           0.411
     ┌───┐┌───┐     ┌─┐
q_0: ┤ H ├┤ X ├─────┤M├
     └───┘└─┬─┘┌───┐└╥┘
q_1: ───────■──┤ X ├─╫─
     ┌───┐┌───┐└───┘ ║ 
q_2: ┤ H ├┤ X ├──────╫─
     └───┘└───┘      ║ 
c: 1/════════════════╩═
                     0 
